In [1]:
%load_ext autoreload
%autoreload 2

### Test DDF against LSST AGN SC DDF Cadence White Paper

In [2]:
# import matplotlib to show plots inline.
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import glob
import os

In [3]:
your_username = os.getcwd().split('/')[5]
print(f'Your automatically extracted username is: {your_username}.'
    ' If it is incorrect, please mannually reset it.')

Your automatically extracted username is: ywx649999311. If it is incorrect, please mannually reset it.


Import the sims_maf modules needed.

In [4]:
# import rubin_sim python modules
import rubin_sim.maf.db as db
import rubin_sim.maf.slicers as slicers
import rubin_sim.maf.metrics as metrics
import rubin_sim.maf.metricBundles as mb
import rubin_sim.maf.plots as plots
import rubin_sim.maf.batches as batches

# import convenience functions for MAF analysis on SciServer
from opsimUtils import *

# print version
import rubin_sim
rubin_sim.__version__

'0.10.1.dev62+gbb0801e'

### **Important:**  
The following code is needed before importing the `opsimUtils` module if the module is not in the same directory as the current notebook. That is, you need add the directory where the `opsimUtils` module is located to the Python search path. 

In [5]:
# add opsimUtils module path to search
import sys
sys.path.insert(0, '../Scripts_NBs/')

# import convenience functions
from opsimUtils import *

#### 1. Build connections to the OpSims databases
The first step is to initiate opsim database objects and result database objects for the opsim databases that you want to run metrics on. Two paths are needed here:
1. `dbDir`: The path to the OpSim database directory
2. `outDir`: The path to the directory where you want to save the metric metadata.

By providing these two paths, the function `connect_dbs()` will initiate connections and store the metadata to the designated directory.

In [6]:
dbRuns = list(run_df.index)
dbRuns[0:5] # only show first 5 opsims

['baseline_retrofoot_v2.0_10yrs',
 'baseline_v2.0_10yrs',
 'baseline_v2.1_10yrs',
 'bluer_indx0_v2.0_10yrs',
 'bluer_indx1_v2.0_10yrs']

In [7]:
if your_username == '': # do NOT put your username here, put it in the cell at the top of the notebook.
    raise Exception('Please provide your username!  See the top of the notebook.')
    
dbDir = rubin_sim.data.get_data_dir()
outDir = '/home/idies/workspace/Storage/{}/persistent/MAFOutput/DDF/'.format(your_username)
metricDataPath = '/home/idies/workspace/Storage/{}/persistent/MAFOutput/DDF/MetricData/'.format(your_username)

if not os.path.exists(os.path.abspath(outDir)):
    os.mkdir(os.path.abspath(outDir))

In [8]:
# two dictionary are returned by the following function, 
# One (opSimDbs) is a dictionary storing all database objects
# Another (resultDbs) is a dictionary consist of the objects directing MAF where to save metric metadata
# Both dictionaries are indexed by OpSim run names
opSimDbs, resultDbs = connect_dbs(dbDir, outDir, dbRuns=dbRuns[0:20])

#### 2. Declare Coadd metrics and run on above OpSims
While constructing a metricBundleGroup from a dictionary (the cell below), you will need to provide the path to a directory (`metricDataPath` in the cell with the nested loops) where you would like to store the metric data (this is **DIFFERENT** than path to the metric data, `outDir`). This path has been declared together with `outDir` in the same cell above. To construct metricbundles for plotting and further analysis, this path will be needed.

In [9]:
bands = ['u', 'g', 'r', 'i', 'z', 'y']
ddfFields = ['COSMOS', 'XMM-LSS', 'ELAISS1', 'ECDFS', 'EDFS']

Since we will need to loop through 5 different DDF fields and 6 different filters, below we design a metric naming covention as `metric_ddfName_band`, where 'metric' is just the original metric name, 'ddfName' is the name of the DDF field (e.g., 'COSMOS') and 'band' is the filter. 

In [10]:
coaddMetricNameTmp = 'coadd_{}_{}'
radius = 1.8
nside = 64

In [11]:
# set summary statistics
summaryMetrics = [metrics.MinMetric(), metrics.MedianMetric(),
                  metrics.MaxMetric(), metrics.RmsMetric()]
# for metric in coaddBDict:
#     coaddBDict[metric].setSummaryMetrics(summaryMetrics)

In [12]:
# loop through opsims
for run in dbRuns[1:5]:
    print(f'Runing metrics on Opsim: {run}!')
    print('*****************************************')
    # one metric bundle dict per opsim
    coaddBDict = {}
    
    # loop through each DDF to get proposalId
    for ddf in ddfFields[0:2]:
        hp_mask = ddf_hp_mask(ddf, radius, nside)
        slicer = slicers.HealpixSubsetSlicer(nside, np.where(hp_mask == 1)[0])
        slicer.slicerName = 'HealpixSlicer'  # hack maf
        for band in bands[0:2]:
            metricName = coaddMetricNameTmp.format(ddf, band)
            coaddMetric = metrics.Coaddm5Metric(metricName=metricName)
            coaddConstraint = 'filter = "{}"'.format(band)
#             coaddConstraint += ' and proposalId = {}'.format(propInfo['proposalId'])
            coaddBDict[metricName] = metricBundles.MetricBundle(coaddMetric, \
                                    slicer, coaddConstraint, runName=run)
            coaddBDict[metricName].setSummaryMetrics(summaryMetrics)

    metricGroup = metricBundles.MetricBundleGroup(coaddBDict, opSimDbs[run], \
                                metricDataPath, resultDbs[run], verbose=False)
    metricGroup.runAll()

Runing metrics on Opsim: baseline_v2.0_10yrs!
*****************************************
Healpix slicer using NSIDE=64, approximate resolution 54.967783 arcminutes
Healpix slicer using NSIDE=64, approximate resolution 54.967783 arcminutes
Runing metrics on Opsim: baseline_v2.1_10yrs!
*****************************************
Healpix slicer using NSIDE=64, approximate resolution 54.967783 arcminutes
Healpix slicer using NSIDE=64, approximate resolution 54.967783 arcminutes
Runing metrics on Opsim: bluer_indx0_v2.0_10yrs!
*****************************************
Healpix slicer using NSIDE=64, approximate resolution 54.967783 arcminutes
Healpix slicer using NSIDE=64, approximate resolution 54.967783 arcminutes
Runing metrics on Opsim: bluer_indx1_v2.0_10yrs!
*****************************************
Healpix slicer using NSIDE=64, approximate resolution 54.967783 arcminutes
Healpix slicer using NSIDE=64, approximate resolution 54.967783 arcminutes


#### 3. Declare NVisits metrics and run on above OpSims

In [13]:
bands = ['u', 'g', 'r', 'i', 'z', 'y']
ddfFields = ['COSMOS', 'XMM-LSS', 'ELAISS1', 'ECDFS', 'EDFS']

In [14]:
nvisitMetricNameTmp = 'nvisit_{}_{}'
radius = 1.8
nside = 64

In [15]:
# set summary statistics
summaryMetrics = [metrics.MinMetric(), metrics.MedianMetric(),
                  metrics.MaxMetric(), metrics.RmsMetric()]
# for metric in coaddBDict:
#     coaddBDict[metric].setSummaryMetrics(summaryMetrics)

In [16]:
# loop through opsims
for run in dbRuns[1:5]:
    print(f'Runing metrics on Opsim: {run}!')
    print('*****************************************')
    # one metric bundle dict per opsim
    nvisitBDict = {}
    
    # loop through each DDF to get proposalId
    for ddf in ddfFields[0:2]:
        hp_mask = ddf_hp_mask(ddf, radius, nside)
        slicer = slicers.HealpixSubsetSlicer(nside, np.where(hp_mask == 1)[0])
        slicer.slicerName = 'HealpixSlicer'  # hack maf
        for band in bands[0:2]:
            metricName = nvisitMetricNameTmp.format(ddf, band)
            nvisitMetric = metrics.CountMetric('observationStartMJD', metricName=metricName)
            nvisitConstraint = 'filter = "{}"'.format(band)
#             nvisitConstraint += ' and proposalId = {}'.format(propInfo['proposalId'])
            nvisitBDict[metricName] = metricBundles.MetricBundle(nvisitMetric, \
                                    slicer, nvisitConstraint, runName=run)
            nvisitBDict[metricName].setSummaryMetrics(summaryMetrics)

    metricGroup = metricBundles.MetricBundleGroup(nvisitBDict, opSimDbs[run], \
                                metricDataPath, resultDbs[run], verbose=False)
    metricGroup.runAll()

Runing metrics on Opsim: baseline_v2.0_10yrs!
*****************************************
Healpix slicer using NSIDE=64, approximate resolution 54.967783 arcminutes
Healpix slicer using NSIDE=64, approximate resolution 54.967783 arcminutes
Runing metrics on Opsim: baseline_v2.1_10yrs!
*****************************************
Healpix slicer using NSIDE=64, approximate resolution 54.967783 arcminutes
Healpix slicer using NSIDE=64, approximate resolution 54.967783 arcminutes
Runing metrics on Opsim: bluer_indx0_v2.0_10yrs!
*****************************************
Healpix slicer using NSIDE=64, approximate resolution 54.967783 arcminutes
Healpix slicer using NSIDE=64, approximate resolution 54.967783 arcminutes
Runing metrics on Opsim: bluer_indx1_v2.0_10yrs!
*****************************************
Healpix slicer using NSIDE=64, approximate resolution 54.967783 arcminutes
Healpix slicer using NSIDE=64, approximate resolution 54.967783 arcminutes
